In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
import copy
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
warnings.filterwarnings(action='ignore')
from epftoolbox.evaluation import MAE, sMAPE

# DNN

In [2]:
df = pd.read_csv(r"Data/df_alemania_final.csv")

In [3]:
df

,Date,Price,Exogenous 1,Exogenous 2,Exogenous 3
0,2019-01-01 00:00:00,28.32,44216.0,0.0,23266.0
1,2019-01-01 01:00:00,10.07,42397.0,0.0,25111.0
2,2019-01-01 02:00:00,-4.08,40788.0,0.0,26717.0
3,2019-01-01 03:00:00,-9.91,40307.0,0.0,28512.0
4,2019-01-01 04:00:00,-7.41,40479.0,0.0,29911.0
...,...,...,...,...,...
38683,2023-05-31 19:00:00,108.26,57968.0,7000.0,13056.0
38684,2023-05-31 20:00:00,119.60,56582.0,2149.0,13078.0
38685,2023-05-31 21:00:00,103.69,54087.0,276.0,14354.0
38686,2023-05-31 22:00:00,94.28,51883.0,0.0,16051.0


In [4]:
df['Date'] = pd.to_datetime(df.Date)

In [5]:
df.loc[df[df['Exogenous 1'].isna()].index, 'Exogenous 1'] = df.loc[df[df['Exogenous 1'].isna()].index - 24, 'Exogenous 1'].to_numpy()
df.loc[df[df['Exogenous 2'].isna()].index, 'Exogenous 2'] = df.loc[df[df['Exogenous 2'].isna()].index - 24, 'Exogenous 2'].to_numpy()
df.loc[df[df['Exogenous 3'].isna()].index, 'Exogenous 3'] = df.loc[df[df['Exogenous 3'].isna()].index - 24, 'Exogenous 3'].to_numpy()

In [6]:
df.isna().sum()

Date           0
Price          0
Exogenous 1    0
Exogenous 2    0
Exogenous 3    0
dtype: int64

In [7]:
df['Exogenous 2'] = df['Exogenous 2'] + df['Exogenous 3']
df = df.drop(['Exogenous 3'], axis = 1)

In [8]:
df.index = df.Date

In [9]:
df = df.drop('Date', axis = 1)

In [10]:
df.to_csv("Data/df_alemania_dnn.csv")

In [11]:
from epftoolbox.models import hyperparameter_optimizer

In [12]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = "df_alemania_dnn"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2022 00:00"
end_test_date = "31/05/2023 23:00"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = 3

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 3

# Number of iterations for hyperparameter optimization
max_evals = 100

path_datasets_folder = "./Data/"
path_hyperparameters_folder = "./experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00


Tested 1/100 iterations.
Best MAE - Validation Dataset
  MAE: 25.5 | sMAPE: 41.18 %

Best MAE - Test Dataset
  MAE: 77.0 | sMAPE: 44.32 %


Tested 2/100 iterations.
Best MAE - Validation Dataset
  MAE: 18.9 | sMAPE: 31.24 %

Best MAE - Test Dataset
  MAE: 194.9 | sMAPE: 45.14 %


Tested 3/100 iterations.
Best MAE - Validation Dataset
  MAE: 18.9 | sMAPE: 31.24 %

Best MAE - Test Dataset
  MAE: 194.9 | sMAPE: 45.14 %


Tested 4/100 iterations.
Best MAE - Validation Dataset
  MAE: 14.4 | sMAPE: 26.54 %

Best MAE - Test Dataset
  MAE: 60.2 | sMAPE: 39.14 %


Tested 5/100 iterations.
Best MAE - Validation Dataset
  MAE: 14.4 | sMAPE: 26.54 %

Best MAE - Test Dataset
  MAE: 60.2 | sMAPE: 39.14 %


Tested 6/100 iterations.
Best MAE - Validation Dataset
  MAE: 14.4 | sMAPE: 26.54 %

Best MAE - Test Dataset
  MAE: 60.2 | sMAPE: 39.14 %


Tested 7/100 iterations.
Best MAE - Validation Dataset
  MAE: 14.4 | sMAPE: 26.54 %

Best MAE - Tes



Tested 59/100 iterations.
Best MAE - Validation Dataset
  MAE: 8.2 | sMAPE: 18.05 %

Best MAE - Test Dataset
  MAE: 38.2 | sMAPE: 27.91 %


Tested 60/100 iterations.
Best MAE - Validation Dataset
  MAE: 8.2 | sMAPE: 18.05 %

Best MAE - Test Dataset
  MAE: 38.2 | sMAPE: 27.91 %


Tested 61/100 iterations.
Best MAE - Validation Dataset
  MAE: 8.2 | sMAPE: 18.05 %

Best MAE - Test Dataset
  MAE: 38.2 | sMAPE: 27.91 %


Tested 62/100 iterations.
Best MAE - Validation Dataset
  MAE: 8.2 | sMAPE: 18.05 %

Best MAE - Test Dataset
  MAE: 38.2 | sMAPE: 27.91 %


Tested 63/100 iterations.
Best MAE - Validation Dataset
  MAE: 8.2 | sMAPE: 18.05 %

Best MAE - Test Dataset
  MAE: 38.2 | sMAPE: 27.91 %


Tested 64/100 iterations.
Best MAE - Validation Dataset
  MAE: 8.2 | sMAPE: 18.05 %

Best MAE - Test Dataset
  MAE: 38.2 | sMAPE: 27.91 %


Tested 65/100 iterations.
Best MAE - Validation Dataset
  MAE: 8.2 | sMAPE: 18.05 %

Best MAE - Test Dataset
  MAE: 38.2 | sMAPE: 27.91 %


Tested 66/100 iter

In [2]:
"""
Example for using the DNN model for forecasting prices with daily recalibration
"""

# Author: Jesus Lago

# License: AGPL-3.0 License

import pandas as pd
import numpy as np
import argparse
import os

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import DNN


path_datasets_folder = "./Data/"
path_hyperparameters_folder = "./experimental_files/"
path_recalibration_folder = os.path.join('.', 'Resultados')
path_hyperparameter_folder = os.path.join('.', 'experimental_files')

dataset = "df_alemania_dnn"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2022 00:00"
end_test_date = "31/05/2023 23:00"

# Defining train and testing data
df_train, df_test = read_data(dataset=dataset, years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

# Defining unique name to save the forecast
forecast_file_name = 'dnn_alemania_final.csv'

forecast_file_path = os.path.join(path_recalibration_folder, forecast_file_name)

# # Defining empty forecast array and the real values to be predicted in a more friendly format
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

# # If we are not starting a new recalibration but re-starting an old one, we import the
# # existing files and print metrics 
# if not new_recalibration:
#     # Import existinf forecasting file
#     forecast = pd.read_csv(forecast_file_path, index_col=0)
#     forecast.index = pd.to_datetime(forecast.index)

#     # Reading dates to still be forecasted by checking NaN values
#     forecast_dates = forecast[forecast.isna().any(axis=1)].index

#     # If all the dates to be forecasted have already been forecast, we print information
#     # and exit the script
#     if len(forecast_dates) == 0:

#         mae = np.mean(MAE(forecast.values.squeeze(), real_values.values))
#         smape = np.mean(sMAPE(forecast.values.squeeze(), real_values.values)) * 100
#         print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format('Final metrics', smape, mae))
    
# else:
#     forecast_dates = forecast.index

forecast_dates = forecast.index

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 3
experiment_id = 3

model = DNN(
    experiment_id=experiment_id, path_hyperparameter_folder=path_hyperparameter_folder, nlayers=nlayers, 
    dataset=dataset, years_test=years_test, shuffle_train=shuffle_train, data_augmentation=data_augmentation,
    calibration_window=calibration_window)


# For loop over the recalibration dates
for date in forecast_dates:

    # For simulation purposes, we assume that the available data is
    # the data up to current date where the prices of current date are not known
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23), :]], axis=0)

    # We extract real prices for current date and set them to NaN in the dataframe of available data
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Recalibrating the model with the most up-to-date available data and making a prediction
    # for the next day
    Yp = model.recalibrate_and_forecast_next_day(df=data_available, next_day_date=date)

    # Saving the current prediction
    forecast.loc[date, :] = Yp

    # Computing metrics up-to-current-date
    mae = np.mean(MAE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) 
    smape = np.mean(sMAPE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) * 100

    # Pringint information
    print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(str(date)[:10], smape, mae))

#     # Saving forecast
#     forecast.to_csv(forecast_file_path)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00
2022-01-01 - sMAPE: 135.10%  |  MAE: 62.471
2022-01-02 - sMAPE: 79.01%  |  MAE: 36.484
2022-01-03 - sMAPE: 86.62%  |  MAE: 39.442
2022-01-04 - sMAPE: 67.99%  |  MAE: 33.868
2022-01-05 - sMAPE: 59.50%  |  MAE: 31.571
2022-01-06 - sMAPE: 51.84%  |  MAE: 30.674
2022-01-07 - sMAPE: 45.13%  |  MAE: 27.363
2022-01-08 - sMAPE: 42.26%  |  MAE: 28.986
2022-01-09 - sMAPE: 38.88%  |  MAE: 27.236
2022-01-10 - sMAPE: 36.71%  |  MAE: 29.841
2022-01-11 - sMAPE: 35.21%  |  MAE: 31.272
2022-01-12 - sMAPE: 32.89%  |  MAE: 29.995
2022-01-13 - sMAPE: 31.19%  |  MAE: 29.120
2022-01-14 - sMAPE: 29.71%  |  MAE: 28.389
2022-01-15 - sMAPE: 28.43%  |  MAE: 27.866
2022-01-16 - sMAPE: 28.00%  |  MAE: 28.059
2022-01-17 - sMAPE: 27.97%  |  MAE: 27.886
2022-01-18 - sMAPE: 27.59%  |  MAE: 29.654
2022-01-19 - sMAPE: 26.66%  |  MAE: 28.820
2022-01-20 - sMAPE: 25.98%  |  MAE: 28.090
2022-01-21 - sMAPE: 25.40%  |  MAE: 27.787
2022-01-22 - sMAPE: 24.62%  |  MAE: 27.

2022-07-10 - sMAPE: 22.15%  |  MAE: 27.533
2022-07-11 - sMAPE: 22.17%  |  MAE: 27.830
2022-07-12 - sMAPE: 22.09%  |  MAE: 27.827
2022-07-13 - sMAPE: 22.03%  |  MAE: 27.853
2022-07-14 - sMAPE: 21.94%  |  MAE: 27.804
2022-07-15 - sMAPE: 21.86%  |  MAE: 27.773
2022-07-16 - sMAPE: 22.13%  |  MAE: 27.903
2022-07-17 - sMAPE: 22.09%  |  MAE: 27.915
2022-07-18 - sMAPE: 22.04%  |  MAE: 28.029
2022-07-19 - sMAPE: 21.99%  |  MAE: 28.134
2022-07-20 - sMAPE: 21.98%  |  MAE: 28.298
2022-07-21 - sMAPE: 21.93%  |  MAE: 28.388
2022-07-22 - sMAPE: 21.84%  |  MAE: 28.318
2022-07-23 - sMAPE: 21.77%  |  MAE: 28.294
2022-07-24 - sMAPE: 21.76%  |  MAE: 28.350
2022-07-25 - sMAPE: 21.71%  |  MAE: 28.374
2022-07-26 - sMAPE: 21.65%  |  MAE: 28.374
2022-07-27 - sMAPE: 21.64%  |  MAE: 28.565
2022-07-28 - sMAPE: 21.56%  |  MAE: 28.572
2022-07-29 - sMAPE: 21.48%  |  MAE: 28.526
2022-07-30 - sMAPE: 21.43%  |  MAE: 28.542
2022-07-31 - sMAPE: 21.38%  |  MAE: 28.556
2022-08-01 - sMAPE: 21.33%  |  MAE: 28.627
2022-08-02 

2023-01-17 - sMAPE: 22.21%  |  MAE: 31.473
2023-01-18 - sMAPE: 22.18%  |  MAE: 31.445
2023-01-19 - sMAPE: 22.17%  |  MAE: 31.424
2023-01-20 - sMAPE: 22.15%  |  MAE: 31.420
2023-01-21 - sMAPE: 22.11%  |  MAE: 31.363
2023-01-22 - sMAPE: 22.07%  |  MAE: 31.307
2023-01-23 - sMAPE: 22.03%  |  MAE: 31.242
2023-01-24 - sMAPE: 21.99%  |  MAE: 31.200
2023-01-25 - sMAPE: 21.96%  |  MAE: 31.182
2023-01-26 - sMAPE: 21.92%  |  MAE: 31.124
2023-01-27 - sMAPE: 21.88%  |  MAE: 31.076
2023-01-28 - sMAPE: 21.85%  |  MAE: 31.029
2023-01-29 - sMAPE: 21.86%  |  MAE: 31.026
2023-01-30 - sMAPE: 21.89%  |  MAE: 30.982
2023-01-31 - sMAPE: 21.90%  |  MAE: 30.970
2023-02-01 - sMAPE: 21.91%  |  MAE: 30.939
2023-02-02 - sMAPE: 21.94%  |  MAE: 30.978
2023-02-03 - sMAPE: 21.97%  |  MAE: 30.978
2023-02-04 - sMAPE: 22.00%  |  MAE: 31.039
2023-02-05 - sMAPE: 21.96%  |  MAE: 30.982
2023-02-06 - sMAPE: 21.92%  |  MAE: 30.944
2023-02-07 - sMAPE: 21.88%  |  MAE: 30.893
2023-02-08 - sMAPE: 21.84%  |  MAE: 30.832
2023-02-09 

In [3]:
forecast

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,1.866798,2.299185,0.023102,0.261187,1.823334,3.999472,1.282951,1.523647,12.558353,22.68269,...,28.824898,31.874973,35.552231,39.83173,36.399059,34.434525,29.390129,26.734648,28.681263,26.309797
2022-01-02,51.446594,43.697601,40.870556,38.994167,32.663242,31.233587,27.815662,32.613487,40.571064,42.161572,...,31.475628,28.488642,33.58123,47.509323,51.804207,46.476059,40.493252,37.957535,39.068665,28.99589
2022-01-03,21.332081,17.09199,15.245903,15.017944,18.146652,25.980026,40.541626,48.657799,52.272476,49.101315,...,42.111572,46.964905,48.802792,54.078186,58.869362,59.37286,51.367653,47.774403,45.787384,40.869083
2022-01-04,91.756302,91.792976,91.549042,90.598152,96.082397,105.325508,139.149078,168.921371,181.843689,180.422913,...,153.914917,151.783508,162.260681,169.741104,166.46759,155.946518,128.839783,110.47374,109.845551,91.235939
2022-01-05,69.967453,57.282455,51.627224,46.377502,46.895977,57.917671,70.840172,102.555641,108.521729,96.728508,...,73.244026,81.069458,94.820213,121.016266,132.486511,128.129959,111.529053,96.931984,99.316483,79.300964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-27,88.948097,77.400635,74.319214,73.710632,74.684013,74.537453,78.416153,82.696884,74.575623,63.30146,...,10.118446,16.065178,30.512066,56.009968,84.067749,103.276573,108.439545,106.24823,98.634674,90.037689
2023-05-28,84.341507,72.232689,71.590652,66.625069,62.791,60.206837,57.614655,53.189606,42.775185,29.715111,...,-27.034065,-25.170204,-12.703102,12.338814,50.514954,67.258682,85.746094,84.048241,84.51358,73.997375
2023-05-29,57.594532,47.707615,41.663872,39.497948,42.115311,43.496567,46.511513,44.266029,37.818237,27.470329,...,-25.867218,-15.62307,1.745422,23.805557,53.248581,76.902618,85.927994,83.744194,81.545563,75.084572


In [4]:
forecast.to_csv("Resultados/dnn_alemania_final.csv")

# DNN + Estandarización móvil

In [2]:
with open('dict_new_df_alemania_final.pkl', 'rb') as handle:
    dict_new_df = pickle.load(handle)

In [3]:
df_nuevo = pd.DataFrame(dict_new_df)

In [4]:
df_nuevo['Date'] = pd.to_datetime(df_nuevo.Date)

In [5]:
df_scalers = pd.DataFrame({'Date':dict_new_df['Date'], 'scaler':dict_new_df['scaler']})

In [6]:
df_scalers

,Date,scaler
0,2019-01-08 00:00:00,StandardScaler()
1,2019-01-08 01:00:00,StandardScaler()
2,2019-01-08 02:00:00,StandardScaler()
3,2019-01-08 03:00:00,StandardScaler()
4,2019-01-08 04:00:00,StandardScaler()
...,...,...
38515,2023-05-31 19:00:00,StandardScaler()
38516,2023-05-31 20:00:00,StandardScaler()
38517,2023-05-31 21:00:00,StandardScaler()
38518,2023-05-31 22:00:00,StandardScaler()


In [7]:
df_nuevo = df_nuevo[['Date', 'Price', 'Exogenous 1', 'Exogenous 2']]

In [12]:
df_nuevo = df_nuevo.set_index('Date')

In [13]:
df_nuevo

,Price,Exogenous 1,Exogenous 2
Date,,,
2019-01-08 00:00:00,-0.783812,-0.208629,1.470236
2019-01-08 01:00:00,-0.676750,-0.398239,1.518663
2019-01-08 02:00:00,-1.150058,-0.540122,1.521090
2019-01-08 03:00:00,-0.913945,-0.511330,1.539723
2019-01-08 04:00:00,-0.761462,-0.432607,1.568935
...,...,...,...
2023-05-31 19:00:00,0.834732,1.036092,-0.143487
2023-05-31 20:00:00,1.051629,0.872712,-0.528166
2023-05-31 21:00:00,0.747323,0.578603,-0.575723


In [14]:
df_nuevo.to_csv("Data/df_alemania_dnn_estandarizacion_movil.csv")

In [15]:
from epftoolbox.models import hyperparameter_optimizer

In [16]:
# Number of layers in DNN
nlayers = 2

# Market under study. If it not one of the standard ones, the file name
# has to be provided, where the file has to be a csv file
dataset = "df_alemania_dnn_estandarizacion_movil"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2022 00:00"
end_test_date = "31/05/2023 23:00"

# Boolean that selects whether the validation and training datasets are shuffled
shuffle_train = 1

# Boolean that selects whether a data augmentation technique for DNNs is used
data_augmentation = 0

# Boolean that selects whether we start a new hyperparameter optimization or we restart an existing one
new_hyperopt = 1

# Number of years used in the training dataset for recalibration
calibration_window = None

# Unique identifier to read the trials file of hyperparameter optimization
experiment_id = 4

# Number of iterations for hyperparameter optimization
max_evals = 100

path_datasets_folder = "./Data/"
path_hyperparameters_folder = "./experimental_files/"

# Check documentation of the hyperparameter_optimizer for each of the function parameters
# In this example, we optimize a model for the PJM market.
# We consider two directories, one for storing the datasets and the other one for the experimental files.
# We start a hyperparameter optimization from scratch. We employ 1500 iterations in hyperopt,
# 2 years of test data, a DNN with 2 hidden layers, a calibration window of 4 years,
# we avoid data augmentation,  and we provide an experiment_id equal to 1
hyperparameter_optimizer(path_datasets_folder=path_datasets_folder, 
                         path_hyperparameters_folder=path_hyperparameters_folder, 
                         new_hyperopt=new_hyperopt, max_evals=max_evals, nlayers=nlayers, dataset=dataset, 
                         years_test=years_test, calibration_window=calibration_window, 
                         shuffle_train=shuffle_train, data_augmentation=0, experiment_id=experiment_id,
                         begin_test_date=begin_test_date, end_test_date=end_test_date)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00


Tested 1/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.8 | sMAPE: 121.91 %

Best MAE - Test Dataset
  MAE: 0.9 | sMAPE: 132.06 %


Tested 2/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.6 | sMAPE: 91.47 %

Best MAE - Test Dataset
  MAE: 0.6 | sMAPE: 98.48 %


Tested 3/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 74.90 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 83.34 %


Tested 4/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 74.90 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 83.34 %


Tested 5/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 74.90 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 83.34 %


Tested 6/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 74.90 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 83.34 %


Tested 7/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 74.90 %

Best MAE - Test Dataset
  M



Tested 60/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.78 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 79.11 %


Tested 61/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.78 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 79.11 %


Tested 62/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.78 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 79.11 %


Tested 63/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.78 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 79.11 %


Tested 64/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.78 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 79.11 %


Tested 65/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 68.78 %

Best MAE - Test Dataset
  MAE: 0.5 | sMAPE: 79.11 %


Tested 66/100 iterations.
Best MAE - Validation Dataset
  MAE: 0.4 | sMAPE: 66.13 %

Best MAE - Test Dataset
  MAE: 0.4 | sMAPE: 75.12 %


Tested 67/100 iterations.

In [17]:
"""
Example for using the DNN model for forecasting prices with daily recalibration
"""

# Author: Jesus Lago

# License: AGPL-3.0 License

import pandas as pd
import numpy as np
import argparse
import os

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import DNN


path_datasets_folder = "./Data/"
path_hyperparameters_folder = "./experimental_files/"
path_recalibration_folder = os.path.join('.', 'Resultados')
path_hyperparameter_folder = os.path.join('.', 'experimental_files')

dataset = "df_alemania_dnn_estandarizacion_movil"

# Number of years (a year is 364 days) in the test dataset.
years_test = 1.5

# Optional parameters for selecting the test dataset, if either of them is not provided, 
# the test dataset is built using the years_test parameter. They should either be one of
# the date formats existing in python or a string with the following format
# "%d/%m/%Y %H:%M"
begin_test_date = "01/01/2022 00:00"
end_test_date = "31/05/2023 23:00"

# Defining train and testing data
df_train, df_test = read_data(dataset=dataset, years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

# Defining unique name to save the forecast
forecast_file_name = 'dnn_alemania_estandarizacion_movil_final.csv'

forecast_file_path = os.path.join(path_recalibration_folder, forecast_file_name)

# # Defining empty forecast array and the real values to be predicted in a more friendly format
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(24)])
real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

# # If we are not starting a new recalibration but re-starting an old one, we import the
# # existing files and print metrics 
# if not new_recalibration:
#     # Import existinf forecasting file
#     forecast = pd.read_csv(forecast_file_path, index_col=0)
#     forecast.index = pd.to_datetime(forecast.index)

#     # Reading dates to still be forecasted by checking NaN values
#     forecast_dates = forecast[forecast.isna().any(axis=1)].index

#     # If all the dates to be forecasted have already been forecast, we print information
#     # and exit the script
#     if len(forecast_dates) == 0:

#         mae = np.mean(MAE(forecast.values.squeeze(), real_values.values))
#         smape = np.mean(sMAPE(forecast.values.squeeze(), real_values.values)) * 100
#         print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format('Final metrics', smape, mae))
    
# else:
#     forecast_dates = forecast.index

forecast_dates = forecast.index

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = None
experiment_id = 4

model = DNN(
    experiment_id=experiment_id, path_hyperparameter_folder=path_hyperparameter_folder, nlayers=nlayers, 
    dataset=dataset, years_test=years_test, shuffle_train=shuffle_train, data_augmentation=data_augmentation,
    calibration_window=calibration_window)


# For loop over the recalibration dates
for date in forecast_dates:

    # For simulation purposes, we assume that the available data is
    # the data up to current date where the prices of current date are not known
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23), :]], axis=0)

    # We extract real prices for current date and set them to NaN in the dataframe of available data
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Recalibrating the model with the most up-to-date available data and making a prediction
    # for the next day
    Yp = model.recalibrate_and_forecast_next_day(df=data_available, next_day_date=date)

    # Saving the current prediction
    forecast.loc[date, :] = Yp

    # Computing metrics up-to-current-date
    mae = np.mean(MAE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) 
    smape = np.mean(sMAPE(forecast.loc[:date].values.squeeze(), real_values.loc[:date].values)) * 100

    # Pringint information
    print('{} - sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(str(date)[:10], smape, mae))

#     # Saving forecast
#     forecast.to_csv(forecast_file_path)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00
2022-01-01 - sMAPE: 58.99%  |  MAE: 0.312
2022-01-02 - sMAPE: 40.56%  |  MAE: 0.244
2022-01-03 - sMAPE: 63.06%  |  MAE: 0.267
2022-01-04 - sMAPE: 58.47%  |  MAE: 0.279
2022-01-05 - sMAPE: 59.75%  |  MAE: 0.268
2022-01-06 - sMAPE: 59.35%  |  MAE: 0.392
2022-01-07 - sMAPE: 57.20%  |  MAE: 0.390
2022-01-08 - sMAPE: 59.44%  |  MAE: 0.399
2022-01-09 - sMAPE: 59.60%  |  MAE: 0.400
2022-01-10 - sMAPE: 57.11%  |  MAE: 0.398
2022-01-11 - sMAPE: 54.76%  |  MAE: 0.381
2022-01-12 - sMAPE: 55.92%  |  MAE: 0.369
2022-01-13 - sMAPE: 58.99%  |  MAE: 0.367
2022-01-14 - sMAPE: 60.51%  |  MAE: 0.361
2022-01-15 - sMAPE: 62.67%  |  MAE: 0.350
2022-01-16 - sMAPE: 61.52%  |  MAE: 0.353
2022-01-17 - sMAPE: 61.54%  |  MAE: 0.363
2022-01-18 - sMAPE: 61.76%  |  MAE: 0.355
2022-01-19 - sMAPE: 62.95%  |  MAE: 0.350
2022-01-20 - sMAPE: 61.90%  |  MAE: 0.345
2022-01-21 - sMAPE: 62.13%  |  MAE: 0.337
2022-01-22 - sMAPE: 63.14%  |  MAE: 0.337
2022-01-23 - sMAPE:

2022-07-14 - sMAPE: 66.70%  |  MAE: 0.377
2022-07-15 - sMAPE: 66.76%  |  MAE: 0.376
2022-07-16 - sMAPE: 66.68%  |  MAE: 0.376
2022-07-17 - sMAPE: 66.56%  |  MAE: 0.375
2022-07-18 - sMAPE: 66.70%  |  MAE: 0.376
2022-07-19 - sMAPE: 66.74%  |  MAE: 0.375
2022-07-20 - sMAPE: 66.77%  |  MAE: 0.375
2022-07-21 - sMAPE: 66.86%  |  MAE: 0.374
2022-07-22 - sMAPE: 66.91%  |  MAE: 0.374
2022-07-23 - sMAPE: 66.89%  |  MAE: 0.373
2022-07-24 - sMAPE: 66.88%  |  MAE: 0.373
2022-07-25 - sMAPE: 66.98%  |  MAE: 0.373
2022-07-26 - sMAPE: 67.05%  |  MAE: 0.374
2022-07-27 - sMAPE: 67.23%  |  MAE: 0.375
2022-07-28 - sMAPE: 67.07%  |  MAE: 0.375
2022-07-29 - sMAPE: 66.91%  |  MAE: 0.374
2022-07-30 - sMAPE: 66.80%  |  MAE: 0.373
2022-07-31 - sMAPE: 66.90%  |  MAE: 0.373
2022-08-01 - sMAPE: 66.80%  |  MAE: 0.373
2022-08-02 - sMAPE: 67.06%  |  MAE: 0.374
2022-08-03 - sMAPE: 67.11%  |  MAE: 0.375
2022-08-04 - sMAPE: 67.03%  |  MAE: 0.374
2022-08-05 - sMAPE: 67.09%  |  MAE: 0.374
2022-08-06 - sMAPE: 67.06%  |  MAE

2023-01-26 - sMAPE: 66.80%  |  MAE: 0.384
2023-01-27 - sMAPE: 66.77%  |  MAE: 0.383
2023-01-28 - sMAPE: 66.79%  |  MAE: 0.383
2023-01-29 - sMAPE: 66.76%  |  MAE: 0.385
2023-01-30 - sMAPE: 66.68%  |  MAE: 0.386
2023-01-31 - sMAPE: 66.69%  |  MAE: 0.386
2023-02-01 - sMAPE: 66.54%  |  MAE: 0.385
2023-02-02 - sMAPE: 66.49%  |  MAE: 0.385
2023-02-03 - sMAPE: 66.60%  |  MAE: 0.385
2023-02-04 - sMAPE: 66.70%  |  MAE: 0.386
2023-02-05 - sMAPE: 66.78%  |  MAE: 0.386
2023-02-06 - sMAPE: 66.66%  |  MAE: 0.385
2023-02-07 - sMAPE: 66.59%  |  MAE: 0.385
2023-02-08 - sMAPE: 66.65%  |  MAE: 0.386
2023-02-09 - sMAPE: 66.74%  |  MAE: 0.386
2023-02-10 - sMAPE: 66.75%  |  MAE: 0.386
2023-02-11 - sMAPE: 66.74%  |  MAE: 0.387
2023-02-12 - sMAPE: 66.80%  |  MAE: 0.386
2023-02-13 - sMAPE: 66.77%  |  MAE: 0.386
2023-02-14 - sMAPE: 66.86%  |  MAE: 0.386
2023-02-15 - sMAPE: 66.86%  |  MAE: 0.386
2023-02-16 - sMAPE: 66.87%  |  MAE: 0.385
2023-02-17 - sMAPE: 66.79%  |  MAE: 0.386
2023-02-18 - sMAPE: 66.82%  |  MAE

In [18]:
forecast

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,-1.730485,-1.908295,-1.937645,-1.857402,-1.825106,-1.818722,-1.553785,-1.190269,-0.853666,-0.633027,...,-0.15897,-0.055646,0.037591,0.305209,0.348652,0.480951,0.078194,-0.3159,-0.332708,-0.655928
2022-01-02,-0.664648,-0.813199,-0.956923,-0.958519,-1.067196,-1.090559,-0.840269,-0.504547,-0.358112,-0.33779,...,-1.119844,-1.077726,-0.938286,-0.640539,-0.577791,-0.628023,-0.878253,-1.038273,-1.082527,-1.430857
2022-01-03,-1.092231,-1.273175,-1.367448,-1.403601,-1.374425,-1.061818,-0.617715,-0.314233,-0.090423,-0.223621,...,-0.16608,-0.049958,0.069075,0.398809,0.403058,0.332699,0.105869,0.004564,-0.090594,-0.358655
2022-01-04,0.193278,0.107856,0.132107,0.017934,0.218181,0.358223,1.112925,1.80279,2.04966,1.954483,...,1.462218,1.569872,1.543018,1.8352,1.687999,1.371304,1.092177,0.643763,0.529729,0.220607
2022-01-05,0.249421,0.121173,0.047117,-0.051445,-0.072515,0.034903,0.425018,0.846814,0.980695,0.87073,...,0.511276,0.518117,0.541266,0.853842,0.89156,0.886658,0.824364,0.723843,0.59007,0.311914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-27,0.393756,0.058979,-0.041917,-0.099748,-0.075632,-0.05529,-0.020875,0.07975,-0.02887,-0.219727,...,-1.448161,-1.318194,-0.862951,-0.237083,0.313849,0.868654,1.164696,1.000257,0.955411,0.69816
2023-05-28,0.480542,0.147445,-0.070023,-0.160519,-0.26953,-0.314175,-0.252486,-0.522808,-0.533655,-0.815443,...,-3.230951,-2.743719,-1.849127,-1.188074,-0.280779,-0.045026,0.412431,0.405828,0.278892,-0.048841
2023-05-29,-0.127669,-0.430784,-0.494225,-0.589769,-0.686904,-0.710326,-0.816641,-0.892395,-1.076527,-1.479828,...,-3.769617,-3.329869,-2.508901,-1.730216,-0.656244,0.050013,0.102022,0.070798,0.134363,-0.174315


In [19]:
forecast.to_csv("Resultados/dnn_alemania_estandarizacion_movil_final.csv")

In [20]:
df_test_original = pd.DataFrame(forecast.values.reshape((-1,)), columns=['Price'])
df_test_original['Date'] = df_scalers.tail(len(df_test_original)).Date.to_numpy()

In [21]:
predicciones = np.array([])
for cont in range(0, len(df_test_original)):
    
    print(df_test_original.iloc[cont].Date)
    
    # Acumulamos las predicciones
    scaler_obj = df_scalers[df_scalers.Date == df_test_original.iloc[cont].Date].scaler.values[0]

    predicciones = np.append(predicciones, scaler_obj.inverse_transform(np.array(df_test_original.iloc[cont].Price).reshape(1, -1))[0][0])

2022-01-01 00:00:00
2022-01-01 01:00:00
2022-01-01 02:00:00
2022-01-01 03:00:00
2022-01-01 04:00:00
2022-01-01 05:00:00
2022-01-01 06:00:00
2022-01-01 07:00:00
2022-01-01 08:00:00
2022-01-01 09:00:00
2022-01-01 10:00:00
2022-01-01 11:00:00
2022-01-01 12:00:00
2022-01-01 13:00:00
2022-01-01 14:00:00
2022-01-01 15:00:00
2022-01-01 16:00:00
2022-01-01 17:00:00
2022-01-01 18:00:00
2022-01-01 19:00:00
2022-01-01 20:00:00
2022-01-01 21:00:00
2022-01-01 22:00:00
2022-01-01 23:00:00
2022-01-02 00:00:00
2022-01-02 01:00:00
2022-01-02 02:00:00
2022-01-02 03:00:00
2022-01-02 04:00:00
2022-01-02 05:00:00
2022-01-02 06:00:00
2022-01-02 07:00:00
2022-01-02 08:00:00
2022-01-02 09:00:00
2022-01-02 10:00:00
2022-01-02 11:00:00
2022-01-02 12:00:00
2022-01-02 13:00:00
2022-01-02 14:00:00
2022-01-02 15:00:00
2022-01-02 16:00:00
2022-01-02 17:00:00
2022-01-02 18:00:00
2022-01-02 19:00:00
2022-01-02 20:00:00
2022-01-02 21:00:00
2022-01-02 22:00:00
2022-01-02 23:00:00
2022-01-03 00:00:00
2022-01-03 01:00:00


2022-01-18 08:00:00
2022-01-18 09:00:00
2022-01-18 10:00:00
2022-01-18 11:00:00
2022-01-18 12:00:00
2022-01-18 13:00:00
2022-01-18 14:00:00
2022-01-18 15:00:00
2022-01-18 16:00:00
2022-01-18 17:00:00
2022-01-18 18:00:00
2022-01-18 19:00:00
2022-01-18 20:00:00
2022-01-18 21:00:00
2022-01-18 22:00:00
2022-01-18 23:00:00
2022-01-19 00:00:00
2022-01-19 01:00:00
2022-01-19 02:00:00
2022-01-19 03:00:00
2022-01-19 04:00:00
2022-01-19 05:00:00
2022-01-19 06:00:00
2022-01-19 07:00:00
2022-01-19 08:00:00
2022-01-19 09:00:00
2022-01-19 10:00:00
2022-01-19 11:00:00
2022-01-19 12:00:00
2022-01-19 13:00:00
2022-01-19 14:00:00
2022-01-19 15:00:00
2022-01-19 16:00:00
2022-01-19 17:00:00
2022-01-19 18:00:00
2022-01-19 19:00:00
2022-01-19 20:00:00
2022-01-19 21:00:00
2022-01-19 22:00:00
2022-01-19 23:00:00
2022-01-20 00:00:00
2022-01-20 01:00:00
2022-01-20 02:00:00
2022-01-20 03:00:00
2022-01-20 04:00:00
2022-01-20 05:00:00
2022-01-20 06:00:00
2022-01-20 07:00:00
2022-01-20 08:00:00
2022-01-20 09:00:00


2022-02-04 10:00:00
2022-02-04 11:00:00
2022-02-04 12:00:00
2022-02-04 13:00:00
2022-02-04 14:00:00
2022-02-04 15:00:00
2022-02-04 16:00:00
2022-02-04 17:00:00
2022-02-04 18:00:00
2022-02-04 19:00:00
2022-02-04 20:00:00
2022-02-04 21:00:00
2022-02-04 22:00:00
2022-02-04 23:00:00
2022-02-05 00:00:00
2022-02-05 01:00:00
2022-02-05 02:00:00
2022-02-05 03:00:00
2022-02-05 04:00:00
2022-02-05 05:00:00
2022-02-05 06:00:00
2022-02-05 07:00:00
2022-02-05 08:00:00
2022-02-05 09:00:00
2022-02-05 10:00:00
2022-02-05 11:00:00
2022-02-05 12:00:00
2022-02-05 13:00:00
2022-02-05 14:00:00
2022-02-05 15:00:00
2022-02-05 16:00:00
2022-02-05 17:00:00
2022-02-05 18:00:00
2022-02-05 19:00:00
2022-02-05 20:00:00
2022-02-05 21:00:00
2022-02-05 22:00:00
2022-02-05 23:00:00
2022-02-06 00:00:00
2022-02-06 01:00:00
2022-02-06 02:00:00
2022-02-06 03:00:00
2022-02-06 04:00:00
2022-02-06 05:00:00
2022-02-06 06:00:00
2022-02-06 07:00:00
2022-02-06 08:00:00
2022-02-06 09:00:00
2022-02-06 10:00:00
2022-02-06 11:00:00


2022-02-21 13:00:00
2022-02-21 14:00:00
2022-02-21 15:00:00
2022-02-21 16:00:00
2022-02-21 17:00:00
2022-02-21 18:00:00
2022-02-21 19:00:00
2022-02-21 20:00:00
2022-02-21 21:00:00
2022-02-21 22:00:00
2022-02-21 23:00:00
2022-02-22 00:00:00
2022-02-22 01:00:00
2022-02-22 02:00:00
2022-02-22 03:00:00
2022-02-22 04:00:00
2022-02-22 05:00:00
2022-02-22 06:00:00
2022-02-22 07:00:00
2022-02-22 08:00:00
2022-02-22 09:00:00
2022-02-22 10:00:00
2022-02-22 11:00:00
2022-02-22 12:00:00
2022-02-22 13:00:00
2022-02-22 14:00:00
2022-02-22 15:00:00
2022-02-22 16:00:00
2022-02-22 17:00:00
2022-02-22 18:00:00
2022-02-22 19:00:00
2022-02-22 20:00:00
2022-02-22 21:00:00
2022-02-22 22:00:00
2022-02-22 23:00:00
2022-02-23 00:00:00
2022-02-23 01:00:00
2022-02-23 02:00:00
2022-02-23 03:00:00
2022-02-23 04:00:00
2022-02-23 05:00:00
2022-02-23 06:00:00
2022-02-23 07:00:00
2022-02-23 08:00:00
2022-02-23 09:00:00
2022-02-23 10:00:00
2022-02-23 11:00:00
2022-02-23 12:00:00
2022-02-23 13:00:00
2022-02-23 14:00:00


2022-03-11 00:00:00
2022-03-11 01:00:00
2022-03-11 02:00:00
2022-03-11 03:00:00
2022-03-11 04:00:00
2022-03-11 05:00:00
2022-03-11 06:00:00
2022-03-11 07:00:00
2022-03-11 08:00:00
2022-03-11 09:00:00
2022-03-11 10:00:00
2022-03-11 11:00:00
2022-03-11 12:00:00
2022-03-11 13:00:00
2022-03-11 14:00:00
2022-03-11 15:00:00
2022-03-11 16:00:00
2022-03-11 17:00:00
2022-03-11 18:00:00
2022-03-11 19:00:00
2022-03-11 20:00:00
2022-03-11 21:00:00
2022-03-11 22:00:00
2022-03-11 23:00:00
2022-03-12 00:00:00
2022-03-12 01:00:00
2022-03-12 02:00:00
2022-03-12 03:00:00
2022-03-12 04:00:00
2022-03-12 05:00:00
2022-03-12 06:00:00
2022-03-12 07:00:00
2022-03-12 08:00:00
2022-03-12 09:00:00
2022-03-12 10:00:00
2022-03-12 11:00:00
2022-03-12 12:00:00
2022-03-12 13:00:00
2022-03-12 14:00:00
2022-03-12 15:00:00
2022-03-12 16:00:00
2022-03-12 17:00:00
2022-03-12 18:00:00
2022-03-12 19:00:00
2022-03-12 20:00:00
2022-03-12 21:00:00
2022-03-12 22:00:00
2022-03-12 23:00:00
2022-03-13 00:00:00
2022-03-13 01:00:00


...

2023-03-13 22:00:00
2023-03-13 23:00:00
2023-03-14 00:00:00
2023-03-14 01:00:00
2023-03-14 02:00:00
2023-03-14 03:00:00
2023-03-14 04:00:00
2023-03-14 05:00:00
2023-03-14 06:00:00
2023-03-14 07:00:00
2023-03-14 08:00:00
2023-03-14 09:00:00
2023-03-14 10:00:00
2023-03-14 11:00:00
2023-03-14 12:00:00
2023-03-14 13:00:00
2023-03-14 14:00:00
2023-03-14 15:00:00
2023-03-14 16:00:00
2023-03-14 17:00:00
2023-03-14 18:00:00
2023-03-14 19:00:00
2023-03-14 20:00:00
2023-03-14 21:00:00
2023-03-14 22:00:00
2023-03-14 23:00:00
2023-03-15 00:00:00
2023-03-15 01:00:00
2023-03-15 02:00:00
2023-03-15 03:00:00
2023-03-15 04:00:00
2023-03-15 05:00:00
2023-03-15 06:00:00
2023-03-15 07:00:00
2023-03-15 08:00:00
2023-03-15 09:00:00
2023-03-15 10:00:00
2023-03-15 11:00:00
2023-03-15 12:00:00
2023-03-15 13:00:00
2023-03-15 14:00:00
2023-03-15 15:00:00
2023-03-15 16:00:00
2023-03-15 17:00:00
2023-03-15 18:00:00
2023-03-15 19:00:00
2023-03-15 20:00:00
2023-03-15 21:00:00
2023-03-15 22:00:00
2023-03-15 23:00:00


2023-03-31 00:00:00
2023-03-31 01:00:00
2023-03-31 02:00:00
2023-03-31 03:00:00
2023-03-31 04:00:00
2023-03-31 05:00:00
2023-03-31 06:00:00
2023-03-31 07:00:00
2023-03-31 08:00:00
2023-03-31 09:00:00
2023-03-31 10:00:00
2023-03-31 11:00:00
2023-03-31 12:00:00
2023-03-31 13:00:00
2023-03-31 14:00:00
2023-03-31 15:00:00
2023-03-31 16:00:00
2023-03-31 17:00:00
2023-03-31 18:00:00
2023-03-31 19:00:00
2023-03-31 20:00:00
2023-03-31 21:00:00
2023-03-31 22:00:00
2023-03-31 23:00:00
2023-04-01 00:00:00
2023-04-01 01:00:00
2023-04-01 02:00:00
2023-04-01 03:00:00
2023-04-01 04:00:00
2023-04-01 05:00:00
2023-04-01 06:00:00
2023-04-01 07:00:00
2023-04-01 08:00:00
2023-04-01 09:00:00
2023-04-01 10:00:00
2023-04-01 11:00:00
2023-04-01 12:00:00
2023-04-01 13:00:00
2023-04-01 14:00:00
2023-04-01 15:00:00
2023-04-01 16:00:00
2023-04-01 17:00:00
2023-04-01 18:00:00
2023-04-01 19:00:00
2023-04-01 20:00:00
2023-04-01 21:00:00
2023-04-01 22:00:00
2023-04-01 23:00:00
2023-04-02 00:00:00
2023-04-02 01:00:00


2023-04-19 03:00:00
2023-04-19 04:00:00
2023-04-19 05:00:00
2023-04-19 06:00:00
2023-04-19 07:00:00
2023-04-19 08:00:00
2023-04-19 09:00:00
2023-04-19 10:00:00
2023-04-19 11:00:00
2023-04-19 12:00:00
2023-04-19 13:00:00
2023-04-19 14:00:00
2023-04-19 15:00:00
2023-04-19 16:00:00
2023-04-19 17:00:00
2023-04-19 18:00:00
2023-04-19 19:00:00
2023-04-19 20:00:00
2023-04-19 21:00:00
2023-04-19 22:00:00
2023-04-19 23:00:00
2023-04-20 00:00:00
2023-04-20 01:00:00
2023-04-20 02:00:00
2023-04-20 03:00:00
2023-04-20 04:00:00
2023-04-20 05:00:00
2023-04-20 06:00:00
2023-04-20 07:00:00
2023-04-20 08:00:00
2023-04-20 09:00:00
2023-04-20 10:00:00
2023-04-20 11:00:00
2023-04-20 12:00:00
2023-04-20 13:00:00
2023-04-20 14:00:00
2023-04-20 15:00:00
2023-04-20 16:00:00
2023-04-20 17:00:00
2023-04-20 18:00:00
2023-04-20 19:00:00
2023-04-20 20:00:00
2023-04-20 21:00:00
2023-04-20 22:00:00
2023-04-20 23:00:00
2023-04-21 00:00:00
2023-04-21 01:00:00
2023-04-21 02:00:00
2023-04-21 03:00:00
2023-04-21 04:00:00


2023-05-08 00:00:00
2023-05-08 01:00:00
2023-05-08 02:00:00
2023-05-08 03:00:00
2023-05-08 04:00:00
2023-05-08 05:00:00
2023-05-08 06:00:00
2023-05-08 07:00:00
2023-05-08 08:00:00
2023-05-08 09:00:00
2023-05-08 10:00:00
2023-05-08 11:00:00
2023-05-08 12:00:00
2023-05-08 13:00:00
2023-05-08 14:00:00
2023-05-08 15:00:00
2023-05-08 16:00:00
2023-05-08 17:00:00
2023-05-08 18:00:00
2023-05-08 19:00:00
2023-05-08 20:00:00
2023-05-08 21:00:00
2023-05-08 22:00:00
2023-05-08 23:00:00
2023-05-09 00:00:00
2023-05-09 01:00:00
2023-05-09 02:00:00
2023-05-09 03:00:00
2023-05-09 04:00:00
2023-05-09 05:00:00
2023-05-09 06:00:00
2023-05-09 07:00:00
2023-05-09 08:00:00
2023-05-09 09:00:00
2023-05-09 10:00:00
2023-05-09 11:00:00
2023-05-09 12:00:00
2023-05-09 13:00:00
2023-05-09 14:00:00
2023-05-09 15:00:00
2023-05-09 16:00:00
2023-05-09 17:00:00
2023-05-09 18:00:00
2023-05-09 19:00:00
2023-05-09 20:00:00
2023-05-09 21:00:00
2023-05-09 22:00:00
2023-05-09 23:00:00
2023-05-10 00:00:00
2023-05-10 01:00:00


2023-05-27 04:00:00
2023-05-27 05:00:00
2023-05-27 06:00:00
2023-05-27 07:00:00
2023-05-27 08:00:00
2023-05-27 09:00:00
2023-05-27 10:00:00
2023-05-27 11:00:00
2023-05-27 12:00:00
2023-05-27 13:00:00
2023-05-27 14:00:00
2023-05-27 15:00:00
2023-05-27 16:00:00
2023-05-27 17:00:00
2023-05-27 18:00:00
2023-05-27 19:00:00
2023-05-27 20:00:00
2023-05-27 21:00:00
2023-05-27 22:00:00
2023-05-27 23:00:00
2023-05-28 00:00:00
2023-05-28 01:00:00
2023-05-28 02:00:00
2023-05-28 03:00:00
2023-05-28 04:00:00
2023-05-28 05:00:00
2023-05-28 06:00:00
2023-05-28 07:00:00
2023-05-28 08:00:00
2023-05-28 09:00:00
2023-05-28 10:00:00
2023-05-28 11:00:00
2023-05-28 12:00:00
2023-05-28 13:00:00
2023-05-28 14:00:00
2023-05-28 15:00:00
2023-05-28 16:00:00
2023-05-28 17:00:00
2023-05-28 18:00:00
2023-05-28 19:00:00
2023-05-28 20:00:00
2023-05-28 21:00:00
2023-05-28 22:00:00
2023-05-28 23:00:00
2023-05-29 00:00:00
2023-05-29 01:00:00
2023-05-29 02:00:00
2023-05-29 03:00:00
2023-05-29 04:00:00
2023-05-29 05:00:00


In [22]:
df_test_original['Price'] = predicciones

In [23]:
df_test_original = df_test_original.set_index(df_test_original.Date).drop('Date', axis = 1)

In [24]:
forecast_final = df_test_original.loc[:, ['Price']].values.reshape(-1, 24)
forecast_final = pd.DataFrame(forecast_final, index=df_test_original.index[::24], columns=['h' + str(k) for k in range(24)])

In [25]:
forecast_final

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,2.261998,-10.199714,-12.256716,-6.632936,-4.369455,-3.922042,14.645953,40.122828,63.713496,79.176864,...,112.401001,119.642381,126.176894,144.932748,147.977432,157.249539,129.022532,101.402613,100.224614,77.571911
2022-01-02,64.690763,55.231869,46.080369,45.978777,39.058849,37.571198,53.508261,74.885100,84.209224,85.503202,...,35.706493,38.388337,47.267081,66.225909,70.221326,67.022827,51.089623,40.900486,38.082620,15.902976
2022-01-03,24.405873,13.681753,8.094428,5.951719,7.680920,26.208396,52.529360,70.516023,83.780697,75.886411,...,79.296685,86.178974,93.233776,112.776376,113.028196,108.858163,95.414510,89.410368,83.770587,67.883263
2022-01-04,89.169521,84.564983,85.872206,79.717871,90.511868,98.060585,138.741472,175.927445,189.234592,184.104206,...,157.569513,163.372408,161.924894,177.674496,169.739858,152.668944,137.623118,113.452148,107.305304,90.642653
2022-01-05,100.705293,93.066878,88.656155,82.785871,81.530949,87.928710,111.163703,136.285686,144.259588,137.710114,...,116.301230,116.708634,118.087420,136.704244,138.950719,138.658776,134.948549,128.961608,120.994128,104.427344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-27,87.286913,73.718598,69.629356,67.285505,68.262895,69.087330,70.482152,74.560428,70.158146,62.422804,...,12.635057,17.902550,36.353290,61.719389,84.048321,106.534248,118.532680,111.868062,110.050463,99.624225
2023-05-28,93.186887,79.605904,70.739333,67.049621,62.605057,60.784773,63.299947,52.278411,51.836160,40.347151,...,-58.137648,-38.272305,-1.798107,25.154287,62.146407,71.758481,90.409882,90.140666,84.965247,71.602938
2023-05-29,67.414421,53.390214,50.454966,46.034451,41.540272,40.456629,35.537718,32.032820,23.513588,4.853996,...,-101.087795,-80.741966,-42.758191,-6.730723,42.958805,75.635248,78.041545,76.596921,79.537881,65.256246


In [26]:
forecast_final.to_csv("Resultados/dnn_alemania_estandarizacion_movil_final.csv")

In [27]:
_, df_test = read_data(dataset="df_alemania_dnn", years_test=years_test, path=path_datasets_folder,
                              begin_test_date=begin_test_date, end_test_date=end_test_date)

real_values = df_test.loc[:, ['Price']].values.reshape(-1, 24)
real_values = pd.DataFrame(real_values, index=forecast.index, columns=forecast.columns)

Test datasets: 2022-01-01 00:00:00 - 2023-05-31 23:00:00


In [28]:
mae = np.mean(MAE(forecast_final.values.squeeze(), real_values.values)) 
smape = np.mean(sMAPE(forecast_final.values.squeeze(), real_values.values)) * 100
print('sMAPE: {:.2f}%  |  MAE: {:.3f}'.format(smape, mae))

sMAPE: 21.53%  |  MAE: 25.400
